# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv").set_index("City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,puerto yuca,2.9383,-73.2083,295.15,97,99,1.00,CO,1727502475
1,puerto natales,-51.7236,-72.4875,273.84,90,72,4.33,CL,1727502299
2,port elizabeth,-33.9180,25.5701,287.77,97,20,3.13,ZA,1727502477
3,grytviken,-54.2811,-36.5092,274.39,93,100,5.84,GS,1727502479
4,port-aux-francais,-49.3500,70.2167,275.26,70,42,13.06,TF,1727502480


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    color="City",
    size=75,
    alpha=0.5,
    frame_width=800,
    frame_height=600,
)

# Display the map
map_plot_1

c:\Users\james\Anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# ~78-80 degrees F
ideal_weather_city_data_df = city_data_df[
    (city_data_df["Max Temp"] > 299) & (city_data_df["Max Temp"] < 300)
]

# Drop any rows with null values
ideal_weather_city_data_df = ideal_weather_city_data_df.dropna()

# Display sample data
ideal_weather_city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
41,kingston,17.9970,-76.7936,299.68,83,20,4.12,JM,1727502348
73,papao,-17.7500,-149.5500,299.97,84,30,7.86,PF,1727502558
74,machina,13.1364,10.0492,299.35,63,82,3.72,NG,1727502559
120,lakhipur,26.0299,90.3041,299.60,85,100,2.47,IN,1727502610
154,mhangura,-16.8939,30.1683,299.98,23,5,3.50,ZW,1727502649
163,kailua-kona,19.6406,-155.9956,299.42,89,72,1.87,US,1727502659
232,tura,25.5198,90.2201,299.96,90,87,1.71,IN,1727502735
235,uturoa,-16.7333,-151.4333,299.73,79,10,6.57,PF,1727502738
240,seminole,28.7086,-81.2081,299.35,79,0,2.57,US,1727502743


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
City_ID,,,,,,,,,,
41,kingston,17.9970,-76.7936,299.68,83,20,4.12,JM,1727502348,
73,papao,-17.7500,-149.5500,299.97,84,30,7.86,PF,1727502558,
74,machina,13.1364,10.0492,299.35,63,82,3.72,NG,1727502559,
120,lakhipur,26.0299,90.3041,299.60,85,100,2.47,IN,1727502610,
154,mhangura,-16.8939,30.1683,299.98,23,5,3.50,ZW,1727502649,
163,kailua-kona,19.6406,-155.9956,299.42,89,72,1.87,US,1727502659,
232,tura,25.5198,90.2201,299.96,90,87,1.71,IN,1727502735,
235,uturoa,-16.7333,-151.4333,299.73,79,10,6.57,PF,1727502738,
240,seminole,28.7086,-81.2081,299.35,79,0,2.57,US,1727502743,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "radius": radius,
    "type": "accommodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kingston - nearest hotel: Aphrodite Hotel
papao - nearest hotel: Hiti Moana Villa Lodge
machina - nearest hotel: No hotel found
lakhipur - nearest hotel: No hotel found
mhangura - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
tura - nearest hotel: Hotel Polo Orchid
uturoa - nearest hotel: Hawaiki Nui hotel
seminole - nearest hotel: Best Western Plus Sanford
kapa'a - nearest hotel: Pono Kai Resort
bubaque - nearest hotel: Chez Julio
bogande - nearest hotel: No hotel found
kokopo - nearest hotel: Hotel Kokopo
naze - nearest hotel: ビジネス旅館畠山
luau - nearest hotel: No hotel found


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
City_ID,,,,,,,,,,
41,kingston,17.9970,-76.7936,299.68,83,20,4.12,JM,1727502348,Aphrodite Hotel
73,papao,-17.7500,-149.5500,299.97,84,30,7.86,PF,1727502558,Hiti Moana Villa Lodge
74,machina,13.1364,10.0492,299.35,63,82,3.72,NG,1727502559,No hotel found
120,lakhipur,26.0299,90.3041,299.60,85,100,2.47,IN,1727502610,No hotel found
154,mhangura,-16.8939,30.1683,299.98,23,5,3.50,ZW,1727502649,No hotel found
163,kailua-kona,19.6406,-155.9956,299.42,89,72,1.87,US,1727502659,Kona Seaside Hotel
232,tura,25.5198,90.2201,299.96,90,87,1.71,IN,1727502735,Hotel Polo Orchid
235,uturoa,-16.7333,-151.4333,299.73,79,10,6.57,PF,1727502738,Hawaiki Nui hotel
240,seminole,28.7086,-81.2081,299.35,79,0,2.57,US,1727502743,Best Western Plus Sanford


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
#%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    color="City",
    size=75,
    alpha=0.5,
    frame_width=800,
    frame_height=600,
    hover_cols=["Lat","Lng","City", "Country", "Hotel Name","Humidity"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name,Humidity)